# Import Libraries and Setup Environment

In [ ]:
import os
import numpy as np
import cv2

from PIL import Image
import binascii as t
import binascii
import codecs
import optparse

import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
os.chdir("/content/drive/My Drive/steganography")
%ls

data/  notebooks/  presentation/  reports/


# Encoder

In [ ]:
file_name = "data/monalisa.jpeg"
directory = "reports"

In [ ]:
message = open("data/message.txt", 'r').read()
message

In [ ]:
def hex_to_rgb(hex_code):
    value = hex_code.lstrip('#')
    lv = len(value)
    return tuple(int(value[i:i + lv // 3], 16) for i in range(0, lv, lv // 3))


def encoder(hex_code, digit):
    if (hex_code[-1] in ('0', '1', '2', '3', '4', '5')):
        hex_code = hex_code[:-1] + digit
        return hex_code
    return None


def rgb_to_hex(r, g, b):
    return "#{:02x}{:02x}{:02x}".format(r, g, b)


def str_to_bin(message):
    binary = bin(int(binascii.hexlify(message.encode("ascii")), 16))
    return binary[2:]


def load_image(file_name):
    image = Image.open(file_name)
    if (image.mode in ("RGBA")):
        image = image.convert("RGBA")
        return image
    return None


def hide(file_name, message, directory):
    img = load_image(file_name)
    if (img != None):
        data = img.getdata()
        binary = str_to_bin(message) + "1111111111111110"
        # print(binary)
        # print(type(binary))
        new_data = []
        digit = 0
        temp = ''
        for d in data:
            if (digit < len(binary)):
                stego_ob = encoder(rgb_to_hex(d[0], d[1], d[2]), binary[digit])
                # print(stego_ob)
                if (stego_ob == None):
                    new_data.append(d)
                else:
                    r, g, b = hex_to_rgb(stego_ob)
                    # print(r, g, b)
                    new_data.append((r, g, b, 255))
                    digit += 1
            else:
                new_data.append(d)
        img.putdata(new_data)
        img.save(os.path.join(directory, file_name.split('/')[-1].split('.')[0]), "PNG")
        return "Hide succefully!" 
    return "Incorrect image mode, couldn't hide"

In [ ]:
hide(file_name, message, directory)

'Hide succefully!'

In [ ]:
img = cv2.imread(file_name)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize = (20,10))
fig = plt.imshow(img)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)
plt.show()

In [ ]:
%ls reports

monalisa


In [ ]:
steganography_image = "reports/monalisa"

In [ ]:
img = cv2.imread(steganography_image)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize = (20,10))
fig = plt.imshow(img)
fig.axes.get_xaxis().set_visible(False)
fig.axes.get_yaxis().set_visible(False)
plt.show()

# Decoder

In [ ]:
def bin_to_str(binary):
    message = t.unhexlify("%x" % int("0b" + binary, 2))
    return message


def decoder(hex_code):
    if (hex_code[-1] in ('0', '1')):
        return hex_code[:-1]
    return None


def retrieving(file_name):
    img = load_image(file_name)
    binary = ''

    if (img != None):
        data = img.getdata()
        for d in data:
            digit = decoder(rgb_to_hex(d[0], d[1], d[2]))
            if (binary[-16:] == "1111111111111110"):
                print("Success!!")
                return bin_to_str(binary[:-16])
        return message

In [ ]:
retrieving(steganography_image)